# CS211: Data Privacy - Final Project
## Haoyuan Pang, Chongqing Gao, Ke Tian

In [85]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
import math
from datetime import datetime

from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# compas_scores_raw = pd.read_csv('https://raw.github.com/g627444300/cs211-final_project/main/compas-scores-raw.csv')

adult = pd.read_csv('https://raw.github.com/g627444300/cs211-final_project/main/cox-violent-parsed.csv')

# cox_violent_parsed_filt = pd.read_csv('https://raw.github.com/g627444300/cs211-final_project/main/cox-violent-parsed_filt.csv')

# df = pd.read_csv('https://raw.github.com/g627444300/cs211-final_project/main/propublica_data_for_fairml.csv')

In [22]:
print (type(adult))
adult.head()

<class 'pandas.core.frame.DataFrame'>


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event
0,1.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,...,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,0,327,0
1,2.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,...,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,334,961,0
2,3.0,michael ryan,michael,ryan,31/12/2014,Male,06/02/1985,31,25 - 45,Caucasian,...,Risk of Violence,2,Low,31/12/2014,30/12/2014,03/01/2015,0,3,457,0
3,4.0,kevon dixon,kevon,dixon,27/01/2013,Male,22/01/1982,34,25 - 45,African-American,...,Risk of Violence,1,Low,27/01/2013,26/01/2013,05/02/2013,0,9,159,1
4,5.0,ed philo,ed,philo,14/04/2013,Male,14/05/1991,24,Less than 25,African-American,...,Risk of Violence,3,Low,14/04/2013,16/06/2013,16/06/2013,4,0,63,0


# dob + laplace

In [68]:
# print(adult.shape)
# display(adult.columns)

In [23]:
epsilon = 1.0

In [24]:
def dob_sum():
    return np.sum(adult['dob'].value_counts() == 1) ## uniquely identified by their age
dob_sum()

3240

In [25]:
def dob_hist():
    return  adult['dob'].value_counts()
dob_hist()

21/11/1991    21
10/11/1988    17
28/04/1992    15
13/08/1994    15
16/10/1983    14
              ..
25/03/1984     1
19/04/1954     1
15/02/1976     1
23/09/1985     1
13/06/1971     1
Name: dob, Length: 7485, dtype: int64

In [26]:
def dp_dob_hist(epsilon):
    return dob_hist().apply(lambda x: laplace_mech(x,1,epsilon))  ## total privacy cost is epsilon, by parallel composition
dp_dob_hist(epsilon)

21/11/1991    17.096817
10/11/1988    17.822079
28/04/1992    14.738579
13/08/1994    13.317947
16/10/1983    11.667165
                ...    
25/03/1984    -0.223739
19/04/1954     3.265226
15/02/1976     0.804344
23/09/1985     0.717240
13/06/1971    -1.116735
Name: dob, Length: 7485, dtype: float64

In [14]:
# def dp_crosstab_education_sex(epsilon):
#     ct = pd.crosstab(adult['dob'], adult['race'])
#     return ct.applymap(lambda x: laplace_mech(x, 1, epsilon))

# dp_crosstab_education_sex(1.0)

print (pct_error(dob_hist(),dp_dob_hist(epsilon)))   #percentage error

21/11/1991      2.929169
10/11/1988      2.749373
28/04/1992     12.984403
13/08/1994     14.371329
27/04/1989      1.022228
                 ...    
28/07/1962     35.810052
14/01/1963    278.729076
22/01/1966    182.181547
06/10/1964     47.748760
17/09/1980    288.419303
Name: dob, Length: 7485, dtype: float64


# # age + laplace

In [55]:
def age_hist():
    return  adult['age'].value_counts()
age_hist() 
# age + laplace
def dp_age_hist(epsilon):
    return age_hist().apply(lambda x: laplace_mech(x,1,epsilon))  ## total privacy cost is epsilon, by parallel composition
dp_age_hist(epsilon)

22    914.908986
26    911.379515
24    910.438248
21    899.889828
25    888.795602
         ...    
78      0.642565
83      2.418694
96      2.630015
80      1.535264
79      0.415801
Name: age, Length: 65, dtype: float64

In [118]:
# age_clipped + laplace
b=90 # age up to 90

real_sum = adult['age'].sum()
def clip_age():
    clipped_age= adult['age'].clip(lower=0, upper=b).sum()
    dp_clipped_age =  age_hist().apply(lambda x: laplace_mech(x,1,epsilon)) #sensitivity = 1
    return (dp_clipped_age)
clip_age()

22    914.295248
26    910.845071
24    911.214475
21    898.497141
25    890.199375
         ...    
78      2.838247
83      1.976059
96      1.776140
80      5.317843
79      1.144874
Name: age, Length: 65, dtype: float64

In [120]:
print (pct_error(age_hist(),clip_age()))   #percentage error

22      0.039909
26      0.010997
24      0.105684
21      0.051925
25      0.052851
         ...    
78     19.035297
83     59.526826
96     20.119115
80    100.223792
79     30.647504
Name: age, Length: 65, dtype: float64


# Compare laplace and Gaussian mechanism in a workload in 'age' set

In [60]:
# Range Query

def range_query(df, col, a, b):
    return len(df[(df[col] >= a) & (df[col] < b)])

random_lower_bounds = [random.randint(1, 70) for _ in range(100)]
random_workload = [(lb, random.randint(lb, 100)) for lb in random_lower_bounds]
real_answers_age = [range_query(adult, 'age', lb, ub) for (lb, ub) in random_workload]
real_answers_start = [range_query(adult, 'start', lb, ub) for (lb, ub) in random_workload]

In [77]:
def workload_laplace(workload, epsilon):
    ans_list = []
    for work in workload:
        a, b = work
        range = range_query(adult, 'age', a, b)
        noised = laplace_mech(range, len(workload), epsilon)
        ans_list.append(noised)
    return ans_list
print('First 4 answers:', workload_laplace(random_workload, 1.0)[:4])
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_age, workload_laplace(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))

First 4 answers: [17852.406735389184, 5089.391032043011, 14114.416543987554, 12324.97410524247]
Average absolute error: 111.79698391384326


In [79]:
def workload_laplace_vec(workload, epsilon):
    ans_list = []
    L1_GS = len(workload)
    range = [range_query(adult, 'age', work[0], work[1]) for work in workload]
    
    noise = laplace_mech_vec(range, L1_GS, epsilon)
    return noise
print('First 4 answers:', workload_laplace_vec(random_workload, 1.0)[:4])
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_age, workload_laplace_vec(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))

First 4 answers: [18350.978392087225, 5366.393237028066, 14138.585395010643, 12210.685270701535]
Average absolute error: 96.88955177890185


In [87]:
def workload_gaussian_vec(workload, epsilon, delta):
    ans_list = []
    L2 = math.sqrt(len(workload))
    rdp = [range_query(adult, 'age', work[0], work[1]) for work in workload]
    
    noise = gaussian_mech_vec(rdp, L2, epsilon, delta)
    return noise
print('First 4 answers:', workload_gaussian_vec(random_workload, 1.0, 1e-5)[:4])
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_age, workload_gaussian_vec(random_workload, 1.0, 1e-5))]
print('Average absolute error:', np.mean(errors))

First 4 answers: [18052.45166833329, 5112.881558247237, 14159.328050693126, 12208.382071446595]
Average absolute error: 38.94582569525386


# # Compare laplace and Gaussian mechanism in a workload in 'start' set


In [83]:
def workload_laplace2(workload, epsilon):
    ans_list = []
    for work in workload:
        a, b = work
        range = range_query(adult, 'start', a, b)
        noised = laplace_mech(range, len(workload), epsilon)
        ans_list.append(noised)
    return ans_list
print('First 4 answers:', workload_laplace2(random_workload, 1.0)[:4])
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_start, workload_laplace2(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))

First 4 answers: [1263.1646049304245, 217.92383365041783, 714.8487421851632, 1497.2123814521242]
Average absolute error: 86.42926870125879


In [81]:
def workload_laplace_vec2(workload, epsilon):
    ans_list = []
    L1_GS = len(workload)
    range = [range_query(adult, 'start', work[0], work[1]) for work in workload]
    
    noise = laplace_mech_vec(range, L1_GS, epsilon)
    return noise
print('First 4 answers:', workload_laplace_vec2(random_workload, 1.0)[:4])
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_start, workload_laplace_vec2(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))

First 4 answers: [1411.21745102895, 117.59281811084422, 850.1828334436161, 1447.5590789578578]
Average absolute error: 99.90407772917624


In [86]:
def workload_gaussian_vec2(workload, epsilon, delta):
    ans_list = []
    L2 = math.sqrt(len(workload))
    rdp = [range_query(adult, 'start', work[0], work[1]) for work in workload]
    
    noise = gaussian_mech_vec(rdp, L2, epsilon, delta)
    return noise
print('First 4 answers:', workload_gaussian_vec2(random_workload, 1.0, 1e-5)[:4])
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_start, workload_gaussian_vec2(random_workload, 1.0, 1e-5))]
print('Average absolute error:', np.mean(errors))

First 4 answers: [1352.9873991165, 282.9076160114672, 749.6842111886693, 1468.5534114422035]
Average absolute error: 36.60416552098758


- Workload_laplace uses sequential composition, and Work_load_laplace_Vec uses L1 sensitivitity. So their accuracy are about the same. 
- Workload_gaussian_vec are the most accurate because it uses L2 sensitivity.
